In [174]:
import pandas as pd
import os
import matplotlib as plt

In [175]:
files = os.listdir('./')
csv_files = [f for f in files if "0.csv" in f]
csv_files

['signals.2018-08-20-060148.0.csv',
 'signals.2018-08-20-065836.0.csv',
 'signals.2018-08-20-075808.0.csv',
 'signals.2018-08-20-113536.0.csv',
 'signals.2018-08-20-092958.0.csv',
 'signals.2018-08-17-222339.0.csv',
 'signals.2018-08-18-063826.0.csv']

In [176]:
frame = pd.DataFrame()
list_ = []
for file_ in csv_files:
    df = pd.read_csv(file_,index_col=None, header=None)
    list_.append(df)
frame = pd.concat(list_)
frame.columns = ["ticker", "price", "time", 
                "meta_signal", "meta_previous",
                "4h","1d","1w","1m",
                "4hp","1dp","1wp","1mp"]
frame.sort_values(['ticker', 'time'], inplace=True)
frame.head(5)

,ticker,price,time,meta_signal,meta_previous,4h,1d,1w,1m,4hp,1dp,1wp,1mp
17,AMEX-BTG,2.160,1534571497681,-1,0,-1,-1.0,-1,-1,0,0.0,0,0
19,AMEX-BTG,2.160,1534604009032,-1,0,-1,-1.0,-1,-1,0,0.0,0,0
44,AMEX-BTG,2.240,1534772816413,-1,0,-1,-2.0,-1,-1,0,-1.0,0,0
16,AMEX-BTG,2.255,1534773899649,-1,0,-1,NaN,-1,-1,0,-2.0,0,0
16,AMEX-BTG,2.275,1534777511412,-1,0,-1,-2.0,-1,-1,0,NaN,0,0


In [177]:
frame.shape

(1032, 13)

In [178]:
# split ticker into market-name
frame['market'], frame['name'] = frame.ticker.str.split('-', 1).str
frame.head(2)

,ticker,price,time,meta_signal,meta_previous,4h,1d,1w,1m,4hp,1dp,1wp,1mp,market,name
17,AMEX-BTG,2.16,1534571497681,-1,0,-1,-1.0,-1,-1,0,0.0,0,0,AMEX,BTG
19,AMEX-BTG,2.16,1534604009032,-1,0,-1,-1.0,-1,-1,0,0.0,0,0,AMEX,BTG


In [180]:
frame.sort_values(['name', 'time'], inplace=True)
frame.head(10)

,ticker,price,time,meta_signal,meta_previous,4h,1d,1w,1m,4hp,1dp,1wp,1mp,market,name
0,NASDAQ-AAPL,217.5800,1534574531372,2,0,1,2.0,2,1,0,0.0,0,0,NASDAQ,AAPL
0,NASDAQ-AAPL,217.5800,1534603573796,2,0,1,2.0,2,1,0,0.0,0,0,NASDAQ,AAPL
28,NASDAQ-AAPL,218.9200,1534772420134,2,1,2,2.0,1,1,1,0.0,2,0,NASDAQ,AAPL
0,NASDAQ-AAPL,217.2198,1534773516623,1,2,1,1.0,1,1,2,2.0,2,0,NASDAQ,AAPL
0,NASDAQ-AAPL,216.0100,1534777088020,1,2,1,1.0,1,1,2,2.0,2,0,NASDAQ,AAPL
155,NASDAQ-AAPL,215.2400,1534781037816,1,2,1,1.0,1,1,2,2.0,2,0,NASDAQ,AAPL
0,NASDAQ-AAPL,215.8831,1534782598765,1,2,1,1.0,1,1,2,2.0,2,0,NASDAQ,AAPL
150,NASDAQ-AAPL,216.4071,1534786163965,1,2,1,1.0,1,1,2,2.0,2,0,NASDAQ,AAPL
0,NASDAQ-AAPL,215.9700,1534790136680,1,2,1,1.0,1,1,2,2.0,2,0,NASDAQ,AAPL
66,NASDAQ-AAPL,215.5950,1534792556346,1,2,1,1.0,1,1,2,2.0,2,0,NASDAQ,AAPL


In [181]:
# failures to get price (numeric conversion)
frame[frame.price.isnull()]

,ticker,price,time,meta_signal,meta_previous,4h,1d,1w,1m,4hp,1dp,1wp,1mp,market,name
148,NASDAQ-VYMI,NaN,1534607161567,-1,0,-1,-2.0,-1,-1,-2,0.0,0,0,NASDAQ,VYMI


In [182]:
frame = frame.dropna()
frame.shape

(1029, 15)

In [183]:
# more failures to get price
frame[frame.price == 0]

,ticker,price,time,meta_signal,meta_previous,4h,1d,1w,1m,4hp,1dp,1wp,1mp,market,name


In [184]:
frame = frame[frame.price!=0]
frame.shape

(1029, 15)

In [185]:
abbv = frame[frame.ticker.astype(str).str.match("NYSE-ABBV")]
abbv

,ticker,price,time,meta_signal,meta_previous,4h,1d,1w,1m,4hp,1dp,1wp,1mp,market,name
1,NYSE-ABBV,98.8100,1534571111519,1,0,1,2.0,1,1,0,0.0,0,0,NYSE,ABBV
1,NYSE-ABBV,98.8100,1534603601535,1,0,1,2.0,1,1,0,0.0,0,0,NYSE,ABBV
29,NYSE-ABBV,99.7101,1534772443104,1,1,2,1.0,1,1,1,2.0,0,0,NYSE,ABBV
1,NYSE-ABBV,100.0400,1534773537603,1,1,2,1.0,1,1,1,2.0,0,0,NYSE,ABBV
1,NYSE-ABBV,98.9900,1534777116671,1,1,1,1.0,1,1,2,2.0,0,0,NYSE,ABBV
156,NYSE-ABBV,98.7300,1534781065462,1,1,1,1.0,1,1,2,2.0,0,0,NYSE,ABBV
1,NYSE-ABBV,98.3600,1534782623958,1,1,2,1.0,1,1,1,2.0,0,0,NYSE,ABBV
151,NYSE-ABBV,98.4700,1534786185517,1,1,1,1.0,1,1,2,2.0,0,0,NYSE,ABBV
1,NYSE-ABBV,98.0100,1534790161756,1,1,1,1.0,1,1,2,2.0,0,0,NYSE,ABBV
67,NYSE-ABBV,97.7600,1534792580524,1,1,1,1.0,1,1,2,2.0,0,0,NYSE,ABBV


In [193]:
# remove duplicates
df = frame.drop_duplicates(["ticker", "price", 
                "meta_signal", "meta_previous",
                "4h","1d","1w","1m",
                "4hp","1dp","1wp","1mp"])
df[df.ticker.astype(str).str.match("NYSE-ABBV")]

,ticker,price,time,meta_signal,meta_previous,4h,1d,1w,1m,4hp,1dp,1wp,1mp,market,name
1,NYSE-ABBV,98.8100,1534571111519,1,0,1,2.0,1,1,0,0.0,0,0,NYSE,ABBV
29,NYSE-ABBV,99.7101,1534772443104,1,1,2,1.0,1,1,1,2.0,0,0,NYSE,ABBV
1,NYSE-ABBV,100.0400,1534773537603,1,1,2,1.0,1,1,1,2.0,0,0,NYSE,ABBV
1,NYSE-ABBV,98.9900,1534777116671,1,1,1,1.0,1,1,2,2.0,0,0,NYSE,ABBV
156,NYSE-ABBV,98.7300,1534781065462,1,1,1,1.0,1,1,2,2.0,0,0,NYSE,ABBV
1,NYSE-ABBV,98.3600,1534782623958,1,1,2,1.0,1,1,1,2.0,0,0,NYSE,ABBV
151,NYSE-ABBV,98.4700,1534786185517,1,1,1,1.0,1,1,2,2.0,0,0,NYSE,ABBV
1,NYSE-ABBV,98.0100,1534790161756,1,1,1,1.0,1,1,2,2.0,0,0,NYSE,ABBV
67,NYSE-ABBV,97.7600,1534792580524,1,1,1,1.0,1,1,2,2.0,0,0,NYSE,ABBV


In [188]:
# test that we have expected number of unique tickers (at least one entry for each)
# currently, we have 159 tickers
print("unique tickers: {} ".format(df.ticker.nunique()))

unique tickers: 159 


In [189]:
# add meta movement indicator
df['meta_move'] = df['meta_signal'].values - df['meta_previous'].values
df.head(3)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ticker,price,time,meta_signal,meta_previous,4h,1d,1w,1m,4hp,1dp,1wp,1mp,market,name,meta_move
0,NASDAQ-AAPL,217.5800,1534574531372,2,0,1,2.0,2,1,0,0.0,0,0,NASDAQ,AAPL,2
28,NASDAQ-AAPL,218.9200,1534772420134,2,1,2,2.0,1,1,1,0.0,2,0,NASDAQ,AAPL,1
0,NASDAQ-AAPL,217.2198,1534773516623,1,2,1,1.0,1,1,2,2.0,2,0,NASDAQ,AAPL,-1


In [190]:
# round to 3 decimals (python tweaks the numbers)
df.price = df.price.round(3)
df[df.name=='HMNY']

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,ticker,price,time,meta_signal,meta_previous,4h,1d,1w,1m,4hp,1dp,1wp,1mp,market,name,meta_move
68,NASDAQ-HMNY,0.029,1534572683953,-1,0,-1,-1.0,-1,-2,0,0.0,0,0,NASDAQ,HMNY,-1
68,NASDAQ-HMNY,0.030,1534778803803,-1,0,-1,-1.0,-1,-2,0,0.0,0,0,NASDAQ,HMNY,-1
67,NASDAQ-HMNY,0.033,1534784171118,-1,0,-1,-1.0,-1,-2,0,0.0,0,0,NASDAQ,HMNY,-1
217,NASDAQ-HMNY,0.031,1534788008309,-1,0,-1,-1.0,-1,-2,0,0.0,0,0,NASDAQ,HMNY,-1


In [191]:
# export new, cleaned db file
df.to_csv('./signals.csv', header=None, index=None)